In [11]:
#import libraries
import pandas as pd
import numpy as np

In [12]:
#Pull in the ratings of the teams and the results of their games
ratings = pd.read_csv("School_stats_files/processed_ratings.csv")
results = pd.read_csv("Spread_and_results_files/processed_results.csv")

In [13]:
#Create another year column, want to put it at the end of the Team1 data
results['YearDup'] = results['Year']

In [14]:
#Move the year column to the eighth index
cols = list(results)
cols.insert(8, cols.pop(cols.index('YearDup')))
results = results.loc[:, cols]

In [15]:
#Remove the 'Unnamed: 0' column
results = results.drop('Unnamed: 0',1)
#results.head()

In [16]:
#Split the data between Team1 results and Team2 results
results2 = pd.DataFrame(results[['Team2',
                                 'Final2',
                                 'Open2',
                                 'Close2',
                                 'ML2',
                                 'Year']])

In [17]:
results.drop(['Team2',
             'Final2',
             'Open2',
             'Close2',
             'ML2',
             'Year'], axis=1, inplace=True)

In [18]:
#Merge the results with the teams' ratings so that each row lists both the team ratings and the results of the game
#Merging on team and year
results_and_ratings = pd.merge(results, ratings,  how='left', left_on=['Team1','YearDup'], right_on = ['School','season_year'])
results_and_ratings2 = pd.merge(results2, ratings,  how='left', left_on=['Team2','Year'], right_on = ['School','season_year'])

In [19]:
#Save in CSV files
results_and_ratings.to_csv('edited_versions/results_and_ratings_exploration.csv')
results_and_ratings2.to_csv('edited_versions/results_and_ratings_exploration2.csv')

In [20]:
#Merge Team1 and Team2 data, save as CSV
spreads_and_ratings_df = pd.merge(results_and_ratings, results_and_ratings2, left_index=True, right_index=True)
spreads_and_ratings_df.to_csv('edited_versions/spreads_and_ratings_df_raw1.csv')

In [21]:
#Drop unnecessary columns
spreads_and_ratings_df.drop(["YearDup","Unnamed: 0_x","School_x","season_year_x",
                        "Unnamed: 0_y","School_y","season_year_y"], axis=1, inplace=True)
#Get a list of columns
cols = list(spreads_and_ratings_df)
#Move the column to head of list using index, pop and insert
cols.insert(2, cols.pop(cols.index('Team2Site')))
spreads_and_ratings_df = spreads_and_ratings_df.loc[:, cols]
spreads_and_ratings_df.to_csv('edited_versions/spreads_and_ratings_df_raw2.csv')

In [22]:
#Rename columns
column_indices = [list(range(8,23))]
Team1_new_names = ['Team1_Conf','Team1_AP_Rank','Team1_W','Team1_L','Team1_OSRS','Team1_DSRS','Team1_SRS',
            'Team1_ScoringOff','Team1_ScoringDef','Team1_PassingOff','Team1_PassingDef','Team1_RushingOff','Team1_RushingDef',
             'Team1_TotalOff','Team1_TotalDef']
Team1_old_names = spreads_and_ratings_df.columns[column_indices]
spreads_and_ratings_df.rename(columns=dict(zip(Team1_old_names, Team1_new_names)), inplace=True)

column_indices = [list(range(29,44))]
Team2_new_names = ['Team2_Conf','Team2_AP_Rank','Team2_W','Team2_L','Team2_OSRS','Team2_DSRS','Team2_SRS',
            'Team2_ScoringOff','Team2_ScoringDef','Team2_PassingOff','Team2_PassingDef','Team2_RushingOff','Team2_RushingDef',
             'Team2_TotalOff','Team2_TotalDef']
Team2_old_names = spreads_and_ratings_df.columns[column_indices]
spreads_and_ratings_df.rename(columns=dict(zip(Team2_old_names, Team2_new_names)), inplace=True)
spreads_and_ratings_df.to_csv('edited_versions/spreads_and_ratings_df_raw3.csv')

In [23]:
#Change spread data to numeric data
spreads_and_ratings_df['Open1'] = pd.to_numeric(spreads_and_ratings_df['Open1'],errors='coerce')
spreads_and_ratings_df['Open2'] = pd.to_numeric(spreads_and_ratings_df['Open2'],errors='coerce')
spreads_and_ratings_df['Close1'] = pd.to_numeric(spreads_and_ratings_df['Close1'],errors='coerce')
spreads_and_ratings_df['Close2'] = pd.to_numeric(spreads_and_ratings_df['Close2'],errors='coerce')

In [24]:
#Create some whether Team1 won label
spreads_and_ratings_df['Team1_Win'] = np.where(spreads_and_ratings_df['Final1']>spreads_and_ratings_df['Final2'], 1, 0)
spreads_and_ratings_df.replace(to_replace="pk", value=0, inplace=True)
#For AP Rank change the blanks to 67(the median)
#Might (will probably) bucket the AP Rankings
spreads_and_ratings_df['Team1_AP_Rank'].fillna(67, inplace=True)
spreads_and_ratings_df['Team2_AP_Rank'].fillna(67, inplace=True)
#Create Team1 and 2 win percentages which should make more sense espescially in early season games
spreads_and_ratings_df['Team1_Win_Precentage'] = spreads_and_ratings_df['Team1_W']/(spreads_and_ratings_df['Team1_W']
                                                                                   + spreads_and_ratings_df['Team1_L'])
spreads_and_ratings_df['Team2_Win_Precentage'] = spreads_and_ratings_df['Team2_W']/(spreads_and_ratings_df['Team2_W']
                                                                                   + spreads_and_ratings_df['Team2_L'])

In [25]:
#Change the No Line instances to the Pandas Null values since it is easier to work with
#Going to just drop as there are not that many and they don't help with predictions
spreads_and_ratings_df.replace(to_replace="NL", value=pd.isnull, inplace=True)
spreads_and_ratings_df.dropna(inplace=True)

#Create the Totals and Spreads features
spreads_and_ratings_df['Total_Open'] = np.where(spreads_and_ratings_df['Open1'] > spreads_and_ratings_df['Open2'], 
                                                spreads_and_ratings_df['Open1'], spreads_and_ratings_df['Open2'])
spreads_and_ratings_df['Total_Close'] = np.where(spreads_and_ratings_df['Close1'] > spreads_and_ratings_df['Close2'], 
                                                spreads_and_ratings_df['Close1'], spreads_and_ratings_df['Close2'])
spreads_and_ratings_df['Spread_Open'] = np.where(spreads_and_ratings_df['Open1'] < spreads_and_ratings_df['Open2'], 
                                                spreads_and_ratings_df['Open1'], spreads_and_ratings_df['Open2'])
spreads_and_ratings_df['Spread_Close'] = np.where(spreads_and_ratings_df['Close1'] < spreads_and_ratings_df['Close2'], 
                                                spreads_and_ratings_df['Close1'], spreads_and_ratings_df['Close2'])

In [26]:
#There were some weird instances in the ML1 and 2 data, deleting those here
spreads_and_ratings_df = spreads_and_ratings_df[pd.to_numeric(spreads_and_ratings_df['ML1'], errors='coerce').notnull()]

#test for non strings
ls = spreads_and_ratings_df.ML1.unique()

#Check for non-strings
for element in ls:
   print(type(element))

spreads_and_ratings_df.to_csv('edited_versions/spreads_and_ratings_df_raw4.csv')

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [27]:
#Converting the spread to their accurate negative values
spreads_and_ratings_df['Spread_Open1'] = np.where(pd.to_numeric(spreads_and_ratings_df['ML1']) < 0, 
                                                -(spreads_and_ratings_df['Spread_Open']), spreads_and_ratings_df['Spread_Open'])
spreads_and_ratings_df['Spread_Close1'] = np.where(pd.to_numeric(spreads_and_ratings_df['ML1']) < 0, 
                                                -(spreads_and_ratings_df['Spread_Close']), 
                                                    spreads_and_ratings_df['Spread_Close'])

In [28]:
#Removing the old spreads and renaming the new ones
spreads_and_ratings_df.drop('Spread_Open',1,inplace=True)
spreads_and_ratings_df.drop('Spread_Close',1,inplace=True)
spreads_and_ratings_df.rename(columns={'Spread_Open1':'Spread_Open','Spread_Close1':'Spread_Close'},inplace=True)

In [29]:
#Creating a point dif column
spreads_and_ratings_df['Team1_Point_Dif'] = spreads_and_ratings_df['Final1'] - spreads_and_ratings_df['Final2']

In [30]:
#Creating whether spread was covered labels
#Template: df.loc[(df['Column_to_compare'] > 18) & (df['Column_to_compare'] < 40), 'Column_to_append'] = '>18'
spreads_and_ratings_df['Team1Cover'] = 0

#favorites scenarios
spreads_and_ratings_df.loc[(spreads_and_ratings_df['Spread_Close'] <=0) & #indicates favorite
                           (spreads_and_ratings_df['Team1_Win'] == 1) & #team had to win
                           (-(spreads_and_ratings_df['Team1_Point_Dif']) < spreads_and_ratings_df['Spread_Close']), #team had to outscore the spread
                           'Team1Cover'] = 1
spreads_and_ratings_df.loc[(spreads_and_ratings_df['Spread_Close'] <=0) & 
                           (spreads_and_ratings_df['Team1_Win'] == 1) &
                           (-(spreads_and_ratings_df['Team1_Point_Dif']) > spreads_and_ratings_df['Spread_Close']),
                           'Team1Cover'] = 0
spreads_and_ratings_df.loc[(spreads_and_ratings_df['Spread_Close'] <=0) & 
                           (spreads_and_ratings_df['Team1_Win'] == 0),
                           'Team1Cover'] = 0

#underdogs scenarios
spreads_and_ratings_df.loc[(spreads_and_ratings_df['Spread_Close'] >=0) &
                           (spreads_and_ratings_df['Team1_Win'] == 0) &
                           (-(spreads_and_ratings_df['Team1_Point_Dif']) > spreads_and_ratings_df['Spread_Close']),
                           'Team1Cover'] = 0
spreads_and_ratings_df.loc[(spreads_and_ratings_df['Spread_Close'] >=0) & 
                           (spreads_and_ratings_df['Team1_Win'] == 1),
                           'Team1Cover'] = 1
spreads_and_ratings_df.loc[(spreads_and_ratings_df['Spread_Close'] >=0) &
                           (spreads_and_ratings_df['Team1_Win'] == 0) &
                           (-(spreads_and_ratings_df['Team1_Point_Dif']) < spreads_and_ratings_df['Spread_Close']),
                           'Team1Cover'] = 1

In [31]:
#Creating the over/under labels
spreads_and_ratings_df['Over_Under'] = 'P'

spreads_and_ratings_df.loc[(spreads_and_ratings_df['Final1'] + spreads_and_ratings_df['Final2'])
                           < (spreads_and_ratings_df['Total_Close']),
                           'Over_Under'] = 'U'
spreads_and_ratings_df.loc[(spreads_and_ratings_df['Final1'] + spreads_and_ratings_df['Final2'])
                           > (spreads_and_ratings_df['Total_Close']),
                           'Over_Under'] = 'O'

In [32]:
spreads_and_ratings_df.to_csv('edited_versions/spreads_and_ratings_df_raw5.csv')

In [33]:
#Changing that god awful date format
#Keeping the month column as a feature
spreads_and_ratings_df['Day'] = spreads_and_ratings_df['Date'].astype(str).str[-2:]
spreads_and_ratings_df['Month'] = spreads_and_ratings_df['Date'].astype(str).str[:-2]
spreads_and_ratings_df['Date'] = (spreads_and_ratings_df['Month'].astype(str))+'/'+(spreads_and_ratings_df['Day'].astype(str))+'/'+(spreads_and_ratings_df['Year'].astype(str))
spreads_and_ratings_df.drop(labels=['Day','Year'], axis=1, inplace=True)

In [34]:
#The conference data includes what sub division they are in within ()s
#Removing these strings, can bring it back if I need to
nums = [1,2]
for team_num in nums:
    spreads_and_ratings_df['Team'+str(team_num)+'_Conf'] = spreads_and_ratings_df['Team'+str(team_num)+'_Conf'].str.replace(r"\(.*\)","")

In [35]:
#Move around some columns

#Get a list of columns
cols = list(spreads_and_ratings_df)
#Move the column to head of list using index, pop and insert
cols.insert(1, cols.pop(cols.index('Month')))
cols.insert(5, cols.pop(cols.index('Team1_Win_Precentage')))
cols.insert(26, cols.pop(cols.index('Team2_Win_Precentage')))
#Insert columns into the DF
spreads_and_ratings_df = spreads_and_ratings_df.loc[:, cols]
spreads_and_ratings_df.head()

,Date,Month,Team1Site,Team2Site,Team1,Team1_Win_Precentage,Final1,Open1,Close1,ML1,...,Team2_TotalOff,Team2_TotalDef,Team1_Win,Total_Open,Total_Close,Spread_Open,Spread_Close,Team1_Point_Dif,Team1Cover,Over_Under
0,9/02/2010,9,H,V,SouthCarolina,0.642857,41,13.5,13.0,-500,...,5.08,6.10,1,48.0,46.5,-13.5,-13.0,28,1,O
1,9/02/2010,9,H,V,OhioState,0.923077,45,28.5,28.0,-5500,...,4.48,5.77,1,47.0,46.5,-28.5,-28.0,38,1,O
2,9/02/2010,9,H,V,IowaState,0.416667,27,3.0,4.0,-190,...,6.08,6.21,1,47.0,51.0,-3.0,-4.0,17,1,U
3,9/02/2010,9,H,V,Utah,0.769231,27,3.5,3.5,-160,...,5.67,5.07,1,50.0,48.0,-3.5,-3.5,3,0,O
4,9/02/2010,9,H,V,Hawaii,0.714286,36,53.5,51.0,752,...,6.07,5.77,0,53.5,51.0,19.0,20.5,-13,1,O


In [36]:
#Put final DF version into a CSV file
spreads_and_ratings_df.to_csv('combined_edited_spreads_and_ratings_df.csv')